In [2]:
import pandas as pd
from ast import literal_eval
import numpy as np
from numpy import nan

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [11]:
ios = pd.read_csv('ios_purchase.csv')
aos = pd.read_csv('aos_purchase.csv')

In [12]:
ios['unique ID'] = ios['IDFA']
aos['unique ID'] = aos['Advertising ID']

In [13]:
df = pd.concat([ios,aos])

In [19]:
df2 = df[~(df['unique ID'].isnull())]

In [20]:
df2['Event Value'] = df2['Event Value'].str.replace('null','nan')
test = df2['Event Value'].values

tmp_list = []

for i in range(len(test)):
    tmp_dict = eval(test[i])
    tmp_id = tmp_dict['challengeIds']
    tmp_list.append(tmp_id)
    
df2['purchase list'] = tmp_list

<ipython-input-20-7d64e54c00f1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Event Value'] = df2['Event Value'].str.replace('null','nan')
<ipython-input-20-7d64e54c00f1>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['purchase list'] = tmp_list


In [21]:
for i in range(len(test)):
    tmp_dict = eval(test[i])
    tmp_id = tmp_dict['challengeIds']
    tmp_list.append(tmp_id)

In [23]:
tmp2 = df2[['Install Time','Event Time','Event Name','unique ID','purchase list']]
tmp2

,Install Time,Event Time,Event Name,unique ID,purchase list
1,2021-01-06 16:18:59,2021-01-06 23:59:40,af_purchase,FD38F906-A676-4934-807D-6E3086626140,[73139]
2,2021-01-06 22:46:20,2021-01-06 23:59:37,af_purchase,543C1DDD-4A0B-495C-8147-EE0EB04E8BA0,[71349]
3,2021-01-06 09:28:51,2021-01-06 23:59:03,af_purchase,82BB54A7-1A8D-4CAE-9970-31A68F25652F,[69835]
5,2020-12-10 02:15:30,2021-01-06 23:57:21,af_purchase,CCA64C4A-6E9B-4D4C-AE0F-CCEDC8093FE3,[73139]
6,2021-01-06 23:46:00,2021-01-06 23:55:18,af_purchase,903FD613-7049-481B-8011-81D85733CF75,[73372]
...,...,...,...,...,...
59117,2020-12-11 09:07:45,2020-12-14 00:00:19,af_purchase,fc05f7a6-4f85-45c1-9656-df5d00fb1b54,[66619]
59118,2020-12-12 13:17:54,2020-12-14 00:00:12,af_purchase,0138314b-eca1-42dc-9599-0e0aeaa7f1d4,[66296]
59119,2020-12-10 23:25:56,2020-12-14 00:00:09,af_purchase,2a90cba8-b9e4-4989-a86e-4e8eff545d53,[66400]
59120,2020-12-13 23:47:21,2020-12-14 00:00:05,af_purchase,73dd50e9-d6c1-47d6-8522-6d083caa53d3,[66427]


In [24]:
user_list = []
purchase_list2 = []

for user in tmp2['unique ID'].unique():
    user_list.append(user)
    
    t1 = tmp2[tmp2['unique ID'] == user]
    tmp_list = []
    
    for list1 in t1['purchase list'].values:
        tmp_list = tmp_list + list1
    uni_list = list(set(tmp_list))
    
    purchase_list2.append(uni_list)

In [25]:
final_df = pd.DataFrame({'ID':user_list, 'purchase content':purchase_list2})
final_df

,ID,purchase content
0,FD38F906-A676-4934-807D-6E3086626140,[73139]
1,543C1DDD-4A0B-495C-8147-EE0EB04E8BA0,[71349]
2,82BB54A7-1A8D-4CAE-9970-31A68F25652F,[69835]
3,CCA64C4A-6E9B-4D4C-AE0F-CCEDC8093FE3,"[69552, 73139, 69660]"
4,903FD613-7049-481B-8011-81D85733CF75,"[72992, 73372]"
...,...,...
17292,d3eecc8c-01a0-48da-9e13-6db1652e3fdd,[66351]
17293,46cbd6ce-8d1f-4250-9a2a-80721c73de3f,[66270]
17294,fc6dc5fb-bbfe-40fe-be12-0cc82a7fe749,[67309]
17295,da330d2b-bed9-4c99-bff6-4bd6ad3df99d,[66425]


In [28]:
df_tmp_list = []
for i in range(len(final_df)):
    df_tmp_list.append(final_df.loc[i,'purchase content'])

In [29]:
df_tmp_arr =  np.array(df_tmp_list)

<ipython-input-29-d4d2a6e22073>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  df_tmp_arr =  np.array(df_tmp_list)


In [33]:
te = TransactionEncoder()
te_ary = te.fit(df_tmp_arr).transform(df_tmp_arr)
result_matrix = pd.DataFrame(te_ary, columns=te.columns_)

In [39]:
frequent_itemsets = apriori(result_matrix, min_support=0.01, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.025496,(65155)
1,0.024339,(65156)
2,0.023588,(65157)
3,0.023530,(65158)
4,0.023299,(65159)
...,...,...
3179,0.010175,"(65158, 65159, 65162, 65163, 65164, 65165)"
3180,0.010117,"(65162, 65163, 65164, 65165, 68218, 68220)"
3181,0.010002,"(65162, 65163, 65164, 65165, 68220, 68221)"
3182,0.010060,"(65163, 65164, 65165, 68218, 68219, 68220)"


In [70]:
final_result = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
final_result

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(65155),(65156),0.025496,0.024339,0.018674,0.732426,30.092109,0.018053,3.646324
1,(65156),(65155),0.024339,0.025496,0.018674,0.767221,30.092109,0.018053,4.186391
2,(65155),(65157),0.025496,0.023588,0.017749,0.696145,29.512800,0.017147,3.213416
3,(65157),(65155),0.023588,0.025496,0.017749,0.752451,29.512800,0.017147,3.936611
4,(65155),(65158),0.025496,0.023530,0.016882,0.662132,28.139776,0.016282,2.890089
...,...,...,...,...,...,...,...,...,...
41521,(65165),"(65164, 68218, 68219, 68220, 68221)",0.020813,0.010522,0.010060,0.483333,45.935256,0.009841,1.915119
41522,(68218),"(65164, 65165, 68219, 68220, 68221)",0.026883,0.010869,0.010060,0.374194,34.427797,0.009767,1.580570
41523,(68219),"(65164, 65165, 68218, 68220, 68221)",0.024339,0.011042,0.010060,0.413302,37.428685,0.009791,1.685632
41524,(68220),"(65164, 65165, 68218, 68219, 68221)",0.025843,0.010695,0.010060,0.389262,36.394921,0.009783,1.619850


In [68]:
## 특정 antecedents만 필터링

final_result[final_result['antecedents'] == frozenset({65155})].sort_values(by='lift',ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
7310,(65155),"(65161, 68218, 65156)",0.025496,0.011042,0.010233,0.401361,36.347295,0.009951,1.652009
24997,(65155),"(65160, 65161, 65156, 65157)",0.025496,0.011389,0.010522,0.412698,36.235759,0.010232,1.683310
25777,(65155),"(65160, 65161, 65156, 65159)",0.025496,0.011331,0.010464,0.410431,36.220522,0.010175,1.676934
7212,(65155),"(65160, 65161, 65156)",0.025496,0.012083,0.011100,0.435374,36.031898,0.010792,1.749684
25537,(65155),"(65160, 65161, 65156, 65158)",0.025496,0.011158,0.010233,0.401361,35.970639,0.009949,1.651816
...,...,...,...,...,...,...,...,...,...
32,(65155),(68221),0.025496,0.023125,0.011910,0.467120,20.199444,0.011320,1.833199
41,(65155),(68227),0.025496,0.024455,0.011042,0.433107,17.710271,0.010419,1.720861
34,(65155),(68222),0.025496,0.022200,0.010002,0.392290,17.670428,0.009436,1.608991
39,(65155),(68226),0.025496,0.025496,0.011447,0.448980,17.609977,0.010797,1.768545


In [64]:
test_list = []
for i in range(len(final_df)):
    test_list += final_df.iloc[i,1]

In [66]:
len(list(set(test_list)))

3767